# Naive Bayes 알고리즘을 활용한 iris 예측

In [2]:
# 필요한 라이브러리 불러오기

from sklearn import datasets 
from sklearn.naive_bayes import GaussianNB  
from sklearn.metrics import confusion_matrix 
import pandas as pd 

## 데이터 로드 및 DataFrame 생성

In [26]:
iris = datasets.load_iris()  # iris 데이터셋 로드
# 특징(feature) 이름을 컬럼으로 지정하여 DataFrame 생성
df_X = pd.DataFrame(iris.data, columns=iris.feature_names)
# 타깃(target) 값을 DataFrame으로 변환
df_Y = pd.DataFrame(iris.target, columns=['target'])

## 데이터 확인 (간단히 출력)

In [27]:
df_X.head()     # 상위 5개 샘플의 특징값 확인  

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [28]:
df_Y.value_counts()  # 각 클래스(0,1,2)가 몇 개씩 존재하는지 집계

target
0         50
1         50
2         50
Name: count, dtype: int64

## 가우시안 나이브 베이즈 분류기 생성

In [29]:
# priors: [클래스0, 클래스1, 클래스2] 순서로 사전 확률 설정
gnb1 = GaussianNB(priors=[0.1, 0.3, 0.6])

## 모델 학습

In [30]:
# fit(X, y): X(입력 특징), y(타깃 레이블)로부터 모델 파라미터 추정
fitted1 = gnb1.fit(iris.data, iris.target)

## 학습 데이터에 대한 예측 (predict)

In [12]:
# predict: 학습 데이터에 대한 클래스 예측
y_pred1 = fitted1.predict(iris.data)

## 혼동 행렬(confusion matrix) 생성

In [13]:
# confusion_matrix: 실제 vs 예측 레이블 비교
cm = confusion_matrix(iris.target, y_pred1)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[50  0  0]
 [ 0 45  5]
 [ 0  3 47]]


## 정확도(Accuracy) 계산 및 출력

In [14]:
# (정답으로 예측된 샘플 수) / (전체 샘플 수) * 100
accuracy = 100 * (iris.target == y_pred1).sum() / iris.data.shape[0]
print("Accuracy = {:.2f}%".format(accuracy))

Accuracy = 94.67%


## 예측 확률 보정 (Calibration) – Sigmoid 방식

In [25]:
from sklearn.calibration import CalibratedClassifierCV  # 확률 보정용 클래스

# cv=2: 2-폴드 교차검증, method='sigmoid': Platt scaling 사용
classifier_sigmoid = CalibratedClassifierCV(gnb1, cv=2, method='sigmoid')
# fit에는 1D array 형태의 y가 필요하므로 .values.ravel() 사용
classifier_sigmoid.fit(df_X, df_Y.values.ravel())


CalibratedClassifierCV(cv=2, estimator=GaussianNB(priors=[0.1, 0.3, 0.6]))

## 새로운 관측치에 대한 보정된 예측 확률 출력

In [24]:
classifier_sigmoid = CalibratedClassifierCV(gnb1, cv=2, method='sigmoid')
classifier_sigmoid.fit(df_X, df_Y.values.ravel())

# 2) 출력은 print()로
new_observation = pd.DataFrame(
    [[6.0, 2.6, 5.0, 1.7]],
    columns=iris.feature_names
)


# predict_proba: 각 클래스에 속할 사후 확률 반환
proba = classifier_sigmoid.predict_proba(new_observation)
print("Calibrated probabilities:", proba)  # 클래스별 보정된 확률 출력



Calibrated probabilities: [[0.02152027 0.27196965 0.70651008]]
